## Lab 8: A/B Testing and Fuzzy Matching ##

In today's lab, we'll take a journey through A/B Testing (a.k.a Split Testing) and Fuzzy Matching

As usual, **run the cell below** to prepare the lab.

In [ ]:
# Run this cell to set up the notebook, but please don't change it.

# These lines import the Numpy and Datascience modules.
import numpy as np
import pandas as pd

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

## 1. Deflategate ##

On January 18, 2015, the Indianapolis Colts and the New England Patriots played the American Football Conference (AFC) championship game to determine which of those teams would play in the Super Bowl. After the game, there were allegations that the Patriots' footballs had not been inflated as much as the regulations required; they were softer. This could be an advantage, as softer balls might be easier to catch.

For several weeks, the world of American football was consumed by accusations, denials, theories, and suspicions: the press labeled the topic Deflategate, after the Watergate political scandal of the 1970's. The National Football League (NFL) commissioned an independent analysis. In this example, we will perform our own analysis of the data.

Pressure is often measured in pounds per square inch (psi). NFL rules stipulate that game balls must be inflated to have pressures in the range 12.5 psi and 13.5 psi. Each team plays with 12 balls. Teams have the responsibility of maintaining the pressure in their own footballs, but game officials inspect the balls. Before the start of the AFC game, all the Patriots' balls were at about 12.5 psi. Most of the Colts' balls were at about 13.0 psi. However, these pre-game data were not recorded.

During the second quarter, the Colts intercepted a Patriots ball. On the sidelines, they measured the pressure of the ball and determined that it was below the 12.5 psi threshold. Promptly, they informed officials.
At half-time, all the game balls were collected for inspection. Two officials, Clete Blakeman and Dyrol Prioleau, measured the pressure in each of the balls.

Here are the data. Each row corresponds to one football. Pressure is measured in psi. The Patriots ball that had been intercepted by the Colts was not inspected at half-time. Nor were most of the Colts’ balls – the officials simply ran out of time and had to relinquish the balls for the start of second half play.


In [ ]:
football = pd.read_csv('deflategate.csv')
football

#### Question 1.1 ####
For each of the 15 balls that were inspected, the two officials got different results. It is not uncommon that repeated measurements on the same object yield different results, especially when the measurements are performed by different people. So we will assign to each the ball the average of the two measurements made on that ball.

In [ ]:
football['Combined'] = ...
football

<details><summary><button>Click here to reveal the answer!</button></summary>
<pre>
football['Combined'] = (football['Blakeman'] + football['Prioleau']) / 2
football
</pre>
</details>

At a glance, it seems apparent that the Patriots' footballs were at a lower pressure than the Colts' balls. Because some deflation is normal during the course of a game, the independent analysts decided to calculate the drop in pressure from the start of the game. Recall that the Patriots' balls had all started out at about 12.5 psi, and the Colts' balls at about 13.0 psi. Therefore the drop in pressure for the Patriots' balls was computed as 12.5 minus the pressure at half-time, and the drop in pressure for the Colts' balls was 13.0 minus the pressure at half-time.

#### Question 1.2 ####

We can calculate the drop in pressure for each football, by first setting up an array of the starting values. For this we will need an array consisting of 11 values each of which is 12.5, and another consisting of four values each of which is all 13. We will use the NumPy function np.ones, which takes a count as its argument and returns an array of that many elements, each of which is 1.

In [ ]:
patriots_start = ...
colts_start = ...
start = ...
start

<details><summary><button>Click here to reveal the answer!</button></summary>
<pre>
patriots_start = 12.5 * np.ones(11)
colts_start = 13 * np.ones(4)
start = np.append(patriots_start, colts_start)
</pre>
</details>

#### Question 1.3 #### 

Let us calcuate the drop in pressure for each football which is the difference between the starting pressure and the combined pressure measurement.

In [ ]:
drop = ...
football['Pressure Drop'] = drop
football

<details><summary><button>Click here to reveal the answer!</button></summary>
<pre>
drop = start - football['Combined']
</pre>
</details>

#### Question 1.4 ####

It looks as though the Patriots' drops were larger than the Colts'. Let's further calculate the average pressure drop in each of the two groups. 

In [ ]:
...

<details><summary><button>Click here to reveal the answer!</button></summary>
<pre>
football.groupby('Team').mean()
</pre>
</details>

The average drop for the Patriots was about 1.2 psi compared to about 0.47 psi for the Colts. 

The question now is why the Patriots' footballs had a larger drop in pressure, on average, than the Colts footballs. Could it be due to chance?

### The Hypotheses ###
How does chance come in here? Nothing was being selected at random. But we can make a chance model by hypothesizing that the 11 Patriots' drops look like a random sample of 11 out of all the 15 drops, with the Colts' drops being the remaining four. That's a completely specified chance model under which we can simulate data. So it's the **null hypothesis**.

For the alternative, we can take the position that the Patriots' drops are too large, on average, to resemble a random sample drawn from all the drops. 

### Test Statistic ###

A natural statistic is the difference between the two average drops, which we will compute as "average drop for Patriots - average drop for Colts". Large values of this statistic will favor the alternative hypothesis.

#### Question 1.5 ####

Calculate the difference between the two average drops.

In [ ]:
observed_means = ...
observed_difference = ...
observed_difference

<details><summary><button>Click here to reveal the answer!</button></summary>
<pre>
observed_means = football.groupby('Team').mean()['Pressure Drop']
observed_difference = observed_means['Patriots'] - observed_means['Colts']
observed_difference
</pre>
</details>

This positive difference reflects the fact that the average drop in pressure of the Patriots' balls was greater than that of the Colts.

### Predicting the Statistic Under the Null Hypothesis ###
If the null hypothesis were true, then the Patriots' drops would be comparable to 11 drops drawn at random without replacement from all 15 drops, and the Colts' drops would be the remaining four.

#### Question 1.6 ####

We can simulate this by randomly permuting all 15 drops and assigning each team the appropriate number of permuted values. Let's do the simulutation and compare the average value

In [ ]:
shuffled_drops = ...
original_and_shuffled = ...
...

<details><summary><button>Click here to reveal the answer!</button></summary>
<pre>
shuffled_drops = football.sample(15, replace=False)['Pressure Drop'].reset_index(drop=True)
original_and_shuffled = football.copy()
original_and_shuffled['Shuffled Drop'] = shuffled_drops
original_and_shuffled.groupby('Team').mean()
</pre>
</details>

The two teams' average drop values are closer when the balls are randomly assigned to the two teams than they were for the balls actually used in the game.


### Permutation Test ###
It's time for a step that is now familiar. We will do repeated simulations of the test statistic under the null hypothesis, by repeatedly permuting the footballs and assigning random sets to the two teams.

#### Question 1.7 ####

We defined a function called `permuted_sample_average_difference` to do this. The code is based on the steps we took to compare the averages of the shuffled data. Use it to compute the average.

In [ ]:
def permuted_sample_average_difference(df, value_column, group_column, repetitions):
    
    tbl = df[[group_column, value_column]]
    tbl_len = len(tbl.index)
    
    differences = []
    for i in np.arange(repetitions):
        shuffled = tbl.sample(tbl_len, replace = False)[value_column].reset_index(drop=True)
        original_and_shuffled = tbl.copy()
        original_and_shuffled['Shuffled Data'] = shuffled

        shuffled_means = original_and_shuffled.groupby(group_column).mean()['Shuffled Data']
        simulated_difference = shuffled_means.iloc[1] - shuffled_means.iloc[0]
        differences = np.append(differences, simulated_difference)
    
    return differences   

In [ ]:
differences = ...
differences

<details><summary><button>Click here to reveal the answer!</button></summary>
<pre>
permuted_sample_average_difference(football, 'Pressure Drop', 'Team', 10000)
</pre>
</details>

### Conclusion of the Test ###
To calculate the empirical P-value, it's important to recall the alternative hypothesis, which is that the Patriots' drops are too large to be the result of chance variation alone.

The "direction of the alternative" is towards large drops for the Patriots, with correspondingly large values for out test statistic "Patriots' average - Colts' average". So the P-value is the chance (computed under the null hypothesis) of getting a test statistic equal to our observed value of 0.73352272727272805 or *larger*.

In [ ]:
empirical_P = np.count_nonzero(differences >= observed_difference) / 10000
empirical_P

That's a pretty small P-value. Let's try to visualize this, here is the empirical distribution of the test statistic under the null hypothesis, with the observed statistic marked on the horizontal axis.

In [ ]:
pd.DataFrame({'Difference Between Group Averages': differences}).plot.hist()
plt.axvline(x=observed_difference, color='red', linestyle='dashed', linewidth='3')
plt.title('Prediction Under the Null Hypothesis')
print('Observed Difference:', observed_difference)
print('Empirical P-value:', empirical_P)

As in previous examples of this test, the bulk of the distribution is centered around 0. Under the null hypothesis, the Patriots' drops are a random sample of all 15 drops, and therefore so are the Colts'. Therefore the two sets of drops should be about equal on average, and therefore their difference should be around 0.

But the observed value of the test statistic is quite far away from the heart of the distribution. By any reasonable cutoff for what is "small", the empirical P-value is small. So we end up rejecting the null hypothesis of randomness, and conclude that the Patriots drops were too large to reflect chance variation alone.

The independent investigative team analyzed the data in several different ways, taking into account the laws of physics. The final report said, 

> "[T]he average pressure drop of the Patriots game balls exceeded the average pressure drop of the Colts balls by 0.45 to 1.02 psi, depending on various possible assumptions regarding the gauges used, and assuming an initial pressure of 12.5 psi for the Patriots balls and 13.0 for the Colts balls."
> 
> -- *Investigative report commissioned by the NFL regarding the AFC Championship game on January 18, 2015*

Our analysis shows an average pressure drop of about 0.73 psi, which is close to the center of the interval "0.45 to 1.02 psi" and therefore consistent with the official analysis.

## 2. Fuzzy String Matching ##

Fuzzy string matching is the process of finding strings that match a given pattern approximately (rather than exactly), like literally. Hence it is also known as approximate string matching. Usually the pattern that these strings are matched against is another string.

The degree of closeness between two strings is measured using Levenshtein Distance, also known as edit distance which basically is based on counting number of primitive operations required to convert one string to the exact match of the other string.

These primitive operations can consist of:

+ Insertion (to insert a new character at a given position)

+ Deletion (to delete a particular character)

+ Substitution (to replace a character with a new one)

+ Transposition (to swap positions of two letters)

### Why Fuzzy Match? ###

Many organizations have been facing the problem of poor data quality that has prevented them from extracting useful customer insights or led to poor business decisions. The primary root cause for poor data quality has been duplicate records for most of the organizations. While it’s fairly straightforward to capture the duplicates which are exact matches, however, spotting the non-exact matches has been a difficult problem to tackle. 

Also, using data which is not duplicate-free organizations tend to initiate poorly performing customer- response driven campaigns or waste of resources to manually identify the approximate matches. 

For example, customer ‘Lisbeth’ who purchased product A according to store database at Location 1 may be same as ‘Lis’ who bought the same product from a different location of store according to store database at Location 2. So this is a case of same customer buying the same product and not different customers with same buying patterns. 

With the advent of fuzzy matching algorithms, it has been possible to identify these hard-to-spot approximate matches. 

### Using FuzzyWuzzy ###

We need `fuzzywuzzy` and `python-Levenshtein` in order to do fuzzy string matching, the following commands will install them on your computer.

`pip install fuzzywuzzy`

`pip install python-Levenshtein`

**Please note that we have already installed these two packages on the jupyterhub (datascience.cs.uchicago.edu). If you're running the lab on your own computer, you have to install the two packages using the command line terminal.**

### Four ways of Fuzzy matching ###

There are four popular types of fuzzy matching logic supported by fuzzywuzzy package:

+ Ratio – uses pure **Levenshtein Distance** based matching 
+ Partial Ratio – matches based on best substrings 
+ Token Sort Ratio – tokenizes the strings and sorts them alphabetically before matching 
+ Token Set Ratio – tokenizes the strings and compared the intersection and remainder 


The code snippets below highlights the difference between these four algorithms with some generic use cases:

First, we import the fuzzywuzzy package

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#### Question 2.1 ####

Let's try to compare a bunch of strings with subtle differences and see how fuzzywuzzy measures them. Run the following commands to test different fuzzy comparisons:

In [ ]:
fuzz.ratio('University of Chicago','University of Chicago!!')

In [ ]:
fuzz.partial_ratio('University of Chicago','university of chicago')

In [ ]:
fuzz.token_set_ratio('University of Chicago','Chicago of University')

In [ ]:
fuzz.token_sort_ratio('University of Chicago','UChicago')

#### Comparing against list of choices ####

The code snippet below demonstrates how you can get scores against a list of choices for a string with any of the four scorers (ratio, partial_ratio, token_sort_ratio, token_set_ratio). The choice of scorer depends on the nature of data and nature of desired results.

In [ ]:
key = "University of Chicago"

# list of 6 string choices to compare against
choices = ['university of chicago','univ. of chicago', 'University of Chicago!', 'Yniversits of Chigago', 'UChicago', 'uchicago']

# Get a list of matches ordered by score, using fuzz.ratio fuzzy matching, default limit to 5
process.extract(key, choices, scorer=fuzz.ratio)

# Get a list of matches ordered by score, using fuzz.ratio fuzzy matching, default limit to 6 (show all)
process.extract(key, choices, scorer=fuzz.ratio, limit = 6)

We can also use “score_cutoff” argument to set a threshold for the best match score. If the best match score is below threshold, it will return “None” as shown in code snippet below.

In [ ]:
# set the threshold for the matching score
process.extractOne(key, choices, scorer=fuzz.ratio, score_cutoff=90)

#### Question 2.2 ####

Define a function to show how fuzzy match can be applied to an entire column of `table_1` to return best score against the column of `table_2`. You can use, for example, the `token_set_ratio` scorer and set the `score_cutoff` as `80`.

In [ ]:
table_1 = pd.DataFrame({'strings': ['University of Chicago', 'Plein Air Cafe', 
                                    'Hyde Park', 'Computer Science']})
table_2 = pd.DataFrame({'strings': ['univ. of chicago', 'Pilsen Air Cafe',
                                   'Hyde Bark', 'Compute Science']})

...
...
...

<details><summary><button>Click here to reveal the answer!</button></summary>
One example of how to do this is below, you can implement the function as you wish
<pre>
def fuzzy_match(x,choices,scorer,cutoff):
    return process.extractOne(
        x, choices=choices,scorer=scorer,score_cutoff=cutoff
    )

FuzzyWuzzyResults=table_1['strings'].apply(
    fuzzy_match,
    args=(table_2['strings'],fuzz.token_set_ratio,80)
)
FuzzyWuzzyResults
</pre>
</details>

Great job! :D You're finished with lab 8!

**Acknowledgement**: The materials for this lab are based on the [data8](http://data8.org/) course at UC Berkeley and [FuzzyWuzzy Using Python](https://www.neudesic.com/blog/fuzzywuzzy-using-python/).